In [2]:
from Bio import SeqIO
import json 
from tqdm import tqdm
import numpy as np 
import pandas as pd

** copying robs work into this directory ** 

In [1]:
!cp -r ../../../../mutation_equilibrium/analysis/coding_v_noncoding_mutability/ ./gettingCdsIntergenic/

In [3]:
model = json.load(open("../Human_mutability_model/Model_2020_12_02_genomeWide.txt"))

**CDS**

In [4]:
cds_count = 0
for record in tqdm(SeqIO.parse(open("gettingCdsIntergenic/cds.nonOverlapping.fasta"), "fasta")):
    cds_count+= 1

209836it [00:01, 123038.27it/s]


In [5]:
print("total CDS sequences = ", cds_count)

total CDS sequences =  209836


In [ ]:
global_cds_mut_list = []
tmp = 0
for record in tqdm(SeqIO.parse(open("gettingCdsIntergenic/cds.nonOverlapping.fasta"), "fasta")): #takes a few mins 
    current_cds_mut_list = []
    for i in range(1, len(record.seq)-1): 
        triplet = str(record.seq[i-1:i+2]).upper()
        mut = model[triplet][0]
        current_cds_mut_list.append(mut)
        tmp+= 1
    global_cds_mut_list.extend(current_cds_mut_list)


11699it [00:04, 2258.89it/s]

In [57]:
np.mean(global_cds_mut_list)

4.2132754899385111e-05

In [8]:
np.mean(global_cds_mut_list)

4.4897492259411877e-05

** intergenic now ** 

In [62]:
print("total intergenic sequences =  ", 30265)

total intergenic sequences =   30265


In [ ]:
global_inter_mut_list = [] #the global list where all the mutabilityies will be appended
for record in tqdm(SeqIO.parse(intergenic_object, "fasta")): #takes maybe an hour or two 
    current_inter_mut_list = [] #makign as sub list for each intergenic region (where all the muts for that region are appended)
    for i in range(1, len(record.seq)-1): #for every centre triplet base in the region: 
        triplet = str(record.seq[i-1:i+2]).upper() #define the triplet ans capitalize 
        if "N" in triplet: 
            continue #skips the rest of the loop for those entires with Ns in them 
        mut = model[triplet][0]  # if no Ns, get the mutability 
        current_inter_mut_list.append(mut) # appedn the mutability to the current list 
    global_inter_mut_list.extend(current_inter_mut_list) #extend the global list by the current (i think this method is fasetr for python, rather than just appeneding t the global each time)


138it [00:06,  6.10it/s]

In [79]:
np.mean(global_inter_mut_list)

3.1678835450268703e-05

** printing to df ** 

In [9]:
data_dict = {"intergenic":[3.1678835450268703e-05, 30265, len(global_inter_mut_list)], "cds":[4.4897492259411877e-05, 209836, len(global_cds_mut_list)]}

NameError: name 'global_inter_mut_list' is not defined

In [82]:
data_df = pd.DataFrame.from_dict(data_dict, orient = "index")

In [101]:
data_df.columns = ["mutability", "n_seqs", "n_sites"]

In [102]:
data_df

,mutability,n_seqs,n_sites
cds,0.000042,1294782,155800440
intergenic,0.000032,30265,1632535849


In [103]:
data_df.to_csv("global_cds_intergenic_mutability.csv")

** testing that my code actually worked ** 

In [84]:
cds_length = 0
for record in SeqIO.parse(open("gettingCdsIntergenic/cds.fasta"), "fasta"): 
    cds_length += len(record.seq)

In [99]:
cds_length, len(global_cds_mut_list), len(global_cds_mut_list) + (2*1294782)

(199915601, 197326285, 199915849)

In [90]:
inter_length = 0
for record in SeqIO.parse(open("gettingCdsIntergenic/intergenic.fasta"), "fasta"): 
    inter_length += len(record.seq)

In [100]:
inter_length, len(global_inter_mut_list)

(1716159126, 1632535849)